(numpyro_conversion)=
# Converting NumPyro objects to DataTree

{class}`~datatree.DataTree` is the data format ArviZ relies on.

This page covers multiple ways to generate a `DataTree` from NumPyro MCMC and SVI objects.

```{seealso}

- Conversion from Python, numpy or pandas objects
- {ref}`working_with_InferenceData` for an overview of `InferenceData` and its role within ArviZ.
```

We will start by importing the required packages and defining the model. The famous 8 school model.

In [19]:
import arviz_base as az
import numpy as np

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, SVI, Trace_ELBO, autoguide, Predictive
from jax import random
import jax.numpy as jnp

In [20]:
J = 8
y_obs = np.array([28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0])
sigma = np.array([15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0])

In [21]:
def eight_schools_model(J, sigma, y=None):
    mu = numpyro.sample("mu", dist.Normal(0, 5))
    tau = numpyro.sample("tau", dist.HalfCauchy(5))
    with numpyro.plate("J", J):
        eta = numpyro.sample("eta", dist.Normal(0, 1))
        theta = numpyro.deterministic("theta", mu + tau * eta)
        return numpyro.sample("obs", dist.Normal(theta, sigma), obs=y)
    

def eight_schools_custom_guide(J, sigma, y=None):

    # Variational parameters for mu
    mu_loc = numpyro.param("mu_loc", 0.0)
    mu_scale = numpyro.param("mu_scale", 1.0, constraint=dist.constraints.positive)
    mu = numpyro.sample("mu", dist.Normal(mu_loc, mu_scale))

    # Variational parameters for tau (positive support)
    tau_loc = numpyro.param("tau_loc", 1.0)
    tau_scale = numpyro.param("tau_scale", 0.5, constraint=dist.constraints.positive)
    tau = numpyro.sample("tau", dist.LogNormal(jnp.log(tau_loc), tau_scale))

    # Variational parameters for eta
    eta_loc = numpyro.param("eta_loc", jnp.zeros(J))
    eta_scale = numpyro.param("eta_scale", jnp.ones(J), constraint=dist.constraints.positive)
    with numpyro.plate("J", J):
        eta = numpyro.sample("eta", dist.Normal(eta_loc, eta_scale))

        # Deterministic transform
        numpyro.deterministic("theta", mu + tau * eta)

## Convert from MCMC

This first example shows conversion from MCMC

In [22]:
# fit with MCMC
nuts = NUTS(eight_schools_model)
mcmc = MCMC(nuts, num_warmup = 1000, num_samples = 1000, num_chains=4)
mcmc.run(random.PRNGKey(0), J=J, sigma=sigma, y=y_obs, extra_fields=("num_steps", "energy"),)

# sample the posterior predictive
predictive = Predictive(eight_schools_model, mcmc.get_samples())
samples_predictive = predictive(random.PRNGKey(1), J=J, sigma=sigma)

# Convert to MCMC
idata_mcmc = az.from_numpyro(mcmc, posterior_predictive=samples_predictive)
idata_mcmc

/var/folders/3n/bm6t53l15kddzf7prg_kj3140000gn/T/ipykernel_47704/3262796440.py:3: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC(nuts, num_warmup = 1000, num_samples = 1000, num_chains=4)
sample: 100%|██████████| 2000/2000 [00:00<00:00, 6662.89it/s, 7 steps of size 3.94e-01. acc. prob=0.90] 


<xarray.DataTree>
Group: /
├── Group: /posterior
│       Dimensions:  (chain: 4, draw: 1000, J: 8)
│       Coordinates:
│         * chain    (chain) int64 32B 0 1 2 3
│         * draw     (draw) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
│         * J        (J) int64 64B 0 1 2 3 4 5 6 7
│       Data variables:
│           eta      (chain, draw, J) float32 128kB 1.691 1.086 ... -0.05226 -0.7434
│           mu       (chain, draw) float32 16kB -3.026 0.3796 8.286 ... 5.022 3.53 1.153
│           tau      (chain, draw) float32 16kB 10.34 4.316 0.6902 ... 1.996 1.01 12.41
│           theta    (chain, draw, J) float32 128kB 14.45 8.198 -6.399 ... 0.5049 -8.071
│       Attributes:
│           created_at:                 2025-10-29T14:25:51.600289+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
├── Group: /sample_stats
│       Dimensions:     (chain: 4, draw: 1000)
│       Coordinates:
│         * chain       (chain) int64 32B 0 1 2 3
│         * draw        (draw) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
│       Data variables:
│           diverging   (chain, draw) bool 4kB ...
│           energy      (chain, draw) float32 16kB ...
│           n_steps     (chain, draw) int32 16kB ...
│           tree_depth  (chain, draw) int64 32kB 4 4 4 3 4 3 3 3 3 ... 3 4 4 4 3 4 3 3 3
│       Attributes:
│           created_at:                 2025-10-29T14:25:51.605378+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
├── Group: /posterior_predictive
│       Dimensions:  (chain: 4, draw: 1000, J: 8)
│       Coordinates:
│         * chain    (chain) int64 32B 0 1 2 3
│         * draw     (draw) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
│         * J        (J) int64 64B 0 1 2 3 4 5 6 7
│       Data variables:
│           obs      (chain, draw, J) float32 128kB 28.67 -1.249 ... 9.282 0.07767
│           theta    (chain, draw, J) float32 128kB 14.45 8.198 -6.399 ... 0.5049 -8.071
│       Attributes:
│           created_at:                 2025-10-29T14:25:51.606625+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
└── Group: /observed_data
        Dimensions:  (J: 8)
        Coordinates:
          * J        (J) int64 64B 0 1 2 3 4 5 6 7
        Data variables:
            obs      (J) float64 64B 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
        Attributes:
            created_at:                 2025-10-29T14:25:51.607351+00:00
            creation_library:           ArviZ
            creation_library_version:   0.7.0.dev0
            creation_library_language:  Python
            inference_library:          numpyro
            inference_library_version:  0.19.0

## Convert from SVI with Autoguide

In [23]:
eight_schools_guide = autoguide.AutoNormal(eight_schools_model, init_loc_fn=numpyro.infer.init_to_median(num_samples=100))
svi = SVI(
    eight_schools_model, 
    guide=eight_schools_guide,
    optim=numpyro.optim.Adam(0.01),
    loss = Trace_ELBO()
)
svi_result = svi.run(random.PRNGKey(0), num_steps=10000, J=J, sigma=sigma, y=y_obs)

# sample the posterior predictive
predictive_svi = Predictive(eight_schools_model, guide=eight_schools_guide, params=svi_result.params, num_samples=4000)
samples_predictive_svi = predictive_svi(random.PRNGKey(1), J=J, sigma=sigma)


idata_svi = az.from_numpyro_svi(
    svi,
    svi_result=svi_result,
    model_kwargs=dict(J=J, sigma=sigma, y=y_obs), # SVI requires providing the fit args/kwargs
    num_samples = 4000, # number of samples to draw in the posterior
    posterior_predictive=samples_predictive_svi
)
idata_svi

100%|██████████| 10000/10000 [00:00<00:00, 10171.06it/s, init loss: 53.6608, avg. loss [9501-10000]: 31.6204]


<xarray.DataTree>
Group: /
├── Group: /posterior
│       Dimensions:  (samples: 4000, J: 8)
│       Coordinates:
│         * samples  (samples) int64 32kB 0 1 2 3 4 5 ... 3994 3995 3996 3997 3998 3999
│         * J        (J) int64 64B 0 1 2 3 4 5 6 7
│       Data variables:
│           eta      (samples, J) float32 128kB -1.212 0.8609 1.496 ... -1.611 -0.6918
│           mu       (samples) float32 16kB -3.658 -2.307 5.14 ... 2.604 0.9337 3.985
│           tau      (samples) float32 16kB 0.9026 1.751 1.003 ... 0.8077 3.972 2.689
│           theta    (samples, J) float32 128kB -4.752 -2.881 -2.308 ... -0.3478 2.125
│       Attributes:
│           created_at:                 2025-10-29T14:25:53.296963+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
├── Group: /sample_stats
│       Attributes:
│           created_at:                 2025-10-29T14:25:53.297860+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
├── Group: /posterior_predictive
│       Dimensions:  (samples: 4000, J: 8)
│       Coordinates:
│         * samples  (samples) int64 32kB 0 1 2 3 4 5 ... 3994 3995 3996 3997 3998 3999
│         * J        (J) int64 64B 0 1 2 3 4 5 6 7
│       Data variables:
│           obs      (samples, J) float32 128kB 4.552 -3.569 -26.3 ... -1.14 -0.9699
│           theta    (samples, J) float32 128kB 2.909 2.912 1.596 ... -3.738 -2.447
│       Attributes:
│           created_at:                 2025-10-29T14:25:53.298510+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
└── Group: /observed_data
        Dimensions:  (J: 8)
        Coordinates:
          * J        (J) int64 64B 0 1 2 3 4 5 6 7
        Data variables:
            obs      (J) float64 64B 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
        Attributes:
            created_at:                 2025-10-29T14:25:53.299119+00:00
            creation_library:           ArviZ
            creation_library_version:   0.7.0.dev0
            creation_library_language:  Python
            inference_library:          numpyro
            inference_library_version:  0.19.0

## Converting from SVI with a custom guide function

In [24]:
svi_custom_guide = SVI(
    eight_schools_model, 
    guide=eight_schools_custom_guide,
    optim=numpyro.optim.Adam(0.01),
    loss = Trace_ELBO()
)
svi_custom_guide_result = svi_custom_guide.run(random.PRNGKey(0), num_steps=10000, J=J, sigma=sigma, y=y_obs)

# sample the posterior predictive
predictive_svi_custom = Predictive(eight_schools_model, guide=eight_schools_custom_guide, params=svi_result.params, num_samples=4000)
samples_predictive_svi_custom = predictive_svi_custom(random.PRNGKey(1), J=J, sigma=sigma)

idata_svi_custom_guide = az.from_numpyro_svi(
    svi_custom_guide,
    svi_result=svi_custom_guide_result,
    model_kwargs=dict(J=J, sigma=sigma, y=y_obs), # SVI requires providing the fit args/kwargs
    num_samples = 4000, # number of samples to draw in the posterior
    posterior_predictive=samples_predictive_svi_custom
)
idata_svi_custom_guide

100%|██████████| 10000/10000 [00:00<00:00, 10440.85it/s, init loss: 34.9525, avg. loss [9501-10000]: 31.6279]


<xarray.DataTree>
Group: /
├── Group: /posterior
│       Dimensions:  (samples: 4000, J: 8)
│       Coordinates:
│         * samples  (samples) int64 32kB 0 1 2 3 4 5 ... 3994 3995 3996 3997 3998 3999
│         * J        (J) int64 64B 0 1 2 3 4 5 6 7
│       Data variables:
│           eta      (samples, J) float32 128kB -2.211 0.1604 1.648 ... -2.492 0.1236
│           mu       (samples) float32 16kB -0.3081 -0.6043 6.962 ... 2.391 6.263 7.293
│           tau      (samples) float32 16kB 0.7794 3.565 4.964 ... 1.445 5.136 2.295
│           theta    (samples, J) float32 128kB -2.032 -0.1831 0.9759 ... 1.575 7.576
│       Attributes:
│           created_at:                 2025-10-29T14:25:54.928034+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
├── Group: /sample_stats
│       Attributes:
│           created_at:                 2025-10-29T14:25:54.929733+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
├── Group: /posterior_predictive
│       Dimensions:  (samples: 4000, J: 8)
│       Coordinates:
│         * samples  (samples) int64 32kB 0 1 2 3 4 5 ... 3994 3995 3996 3997 3998 3999
│         * J        (J) int64 64B 0 1 2 3 4 5 6 7
│       Data variables:
│           obs      (samples, J) float32 128kB 0.8301 -7.256 -29.28 ... -0.2221 -0.507
│           theta    (samples, J) float32 128kB -0.813 -0.7758 -1.391 ... -2.82 -1.984
│       Attributes:
│           created_at:                 2025-10-29T14:25:54.932150+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
└── Group: /observed_data
        Dimensions:  (J: 8)
        Coordinates:
          * J        (J) int64 64B 0 1 2 3 4 5 6 7
        Data variables:
            obs      (J) float64 64B 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
        Attributes:
            created_at:                 2025-10-29T14:25:54.934490+00:00
            creation_library:           ArviZ
            creation_library_version:   0.7.0.dev0
            creation_library_language:  Python
            inference_library:          numpyro
            inference_library_version:  0.19.0

## Automatically Labelling Event Dims

NumPyro batch dims are automatically labelled according to their corresponding plate names. In order to label event dims, we can do the following:

In [25]:
def eight_schools_model_zsn(J, sigma, y=None):
    mu = numpyro.sample("mu", dist.Normal(0, 5))
    tau = numpyro.sample("tau", dist.HalfCauchy(5))
    eta = numpyro.sample(
        "eta", 
        dist.ZeroSumNormal(tau, event_shape=(J,)), 
        infer={"event_dims":["groups"]}
    )
    with numpyro.plate("J", J):
        theta = numpyro.deterministic("theta", mu + eta)
        return numpyro.sample("obs", dist.Normal(theta, sigma), obs=y)
    

# fit with MCMC
nuts = NUTS(eight_schools_model_zsn)
mcmc2 = MCMC(nuts, num_warmup = 1000, num_samples = 1000, num_chains=4)
mcmc2.run(random.PRNGKey(0), J=J, sigma=sigma, y=y_obs, extra_fields=("num_steps", "energy"),)


# sample the posterior predictive
predictive2 = Predictive(eight_schools_model, mcmc2.get_samples())
samples_predictive2 = predictive2(random.PRNGKey(1), J=J, sigma=sigma)

# Convert to MCMC
idata_mcmc2 = az.from_numpyro(mcmc, posterior_predictive=samples_predictive2)

# Notice theta is labelled appropriately with `J`
idata_mcmc2

/var/folders/3n/bm6t53l15kddzf7prg_kj3140000gn/T/ipykernel_47704/1587679241.py:16: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc2 = MCMC(nuts, num_warmup = 1000, num_samples = 1000, num_chains=4)
sample: 100%|██████████| 2000/2000 [00:00<00:00, 6872.46it/s, 31 steps of size 1.85e-01. acc. prob=0.91]


<xarray.DataTree>
Group: /
├── Group: /posterior
│       Dimensions:  (chain: 4, draw: 1000, J: 8)
│       Coordinates:
│         * chain    (chain) int64 32B 0 1 2 3
│         * draw     (draw) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
│         * J        (J) int64 64B 0 1 2 3 4 5 6 7
│       Data variables:
│           eta      (chain, draw, J) float32 128kB 1.691 1.086 ... -0.05226 -0.7434
│           mu       (chain, draw) float32 16kB -3.026 0.3796 8.286 ... 5.022 3.53 1.153
│           tau      (chain, draw) float32 16kB 10.34 4.316 0.6902 ... 1.996 1.01 12.41
│           theta    (chain, draw, J) float32 128kB 14.45 8.198 -6.399 ... 0.5049 -8.071
│       Attributes:
│           created_at:                 2025-10-29T14:25:56.644669+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
├── Group: /sample_stats
│       Dimensions:     (chain: 4, draw: 1000)
│       Coordinates:
│         * chain       (chain) int64 32B 0 1 2 3
│         * draw        (draw) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
│       Data variables:
│           diverging   (chain, draw) bool 4kB ...
│           energy      (chain, draw) float32 16kB ...
│           n_steps     (chain, draw) int32 16kB ...
│           tree_depth  (chain, draw) int64 32kB 4 4 4 3 4 3 3 3 3 ... 3 4 4 4 3 4 3 3 3
│       Attributes:
│           created_at:                 2025-10-29T14:25:56.646299+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
├── Group: /posterior_predictive
│       Dimensions:  (chain: 4, draw: 1000, J: 8)
│       Coordinates:
│         * chain    (chain) int64 32B 0 1 2 3
│         * draw     (draw) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
│         * J        (J) int64 64B 0 1 2 3 4 5 6 7
│       Data variables:
│           obs      (chain, draw, J) float32 128kB 22.29 10.91 -11.63 ... 6.396 5.612
│           theta    (chain, draw, J) float32 128kB 8.072 20.36 2.025 ... -2.381 -2.537
│       Attributes:
│           created_at:                 2025-10-29T14:25:56.647836+00:00
│           creation_library:           ArviZ
│           creation_library_version:   0.7.0.dev0
│           creation_library_language:  Python
│           inference_library:          numpyro
│           inference_library_version:  0.19.0
└── Group: /observed_data
        Dimensions:  (J: 8)
        Coordinates:
          * J        (J) int64 64B 0 1 2 3 4 5 6 7
        Data variables:
            obs      (J) float64 64B 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
        Attributes:
            created_at:                 2025-10-29T14:25:56.648629+00:00
            creation_library:           ArviZ
            creation_library_version:   0.7.0.dev0
            creation_library_language:  Python
            inference_library:          numpyro
            inference_library_version:  0.19.0

In [28]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Wed Oct 29 2025

Python implementation: CPython
Python version       : 3.12.10
IPython version      : 9.4.0

numpy     : 2.3.2
jax       : 0.6.2
arviz_base: 0.7.0.dev0
numpyro   : 0.19.0

Watermark: 2.5.0

